In [25]:
!apt-get update
!apt-get install ffmpeg libsm6 libxext6 -y
!python -m pip install opencv-python

python is /usr/local/bin/python


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten ,Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential
import numpy as np
import os 
import random
import cv2
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
#from sklearn.model_selection import train_test_split

In [7]:
rootDir = './animals-dataset'
trainingDir = rootDir + '/training'

In [13]:
def getData(path):
    foldernames = os.listdir(path)
    categories = {}
    
    for k, folder in enumerate(foldernames):
        categories[folder] = list()
        files = os.listdir(path + '/' + folder)
        for i, file in enumerate(files):
            img = cv2.imread(path + '/' + folder + '/' + file)
            categories[folder].append(img)
    return categories

In [18]:
trainingData = getData(trainingDir)

NotADirectoryError: [Errno 20] Not a directory: './.gitignore'

In [19]:
print(trainingData['dogs'])

[array([[[162, 167, 170],
        [154, 159, 162],
        [149, 153, 158],
        ...,
        [213, 229, 236],
        [215, 229, 235],
        [217, 229, 233]],

       [[156, 161, 164],
        [147, 152, 155],
        [143, 147, 152],
        ...,
        [201, 217, 224],
        [203, 217, 223],
        [203, 217, 223]],

       [[152, 157, 160],
        [143, 148, 151],
        [137, 143, 148],
        ...,
        [197, 215, 226],
        [198, 215, 224],
        [200, 215, 224]],

       ...,

       [[218, 218, 232],
        [214, 214, 228],
        [208, 208, 222],
        ...,
        [213, 220, 229],
        [214, 221, 230],
        [215, 222, 231]],

       [[219, 220, 234],
        [215, 216, 230],
        [210, 211, 225],
        ...,
        [222, 228, 235],
        [223, 229, 236],
        [224, 230, 237]],

       [[228, 229, 243],
        [224, 225, 239],
        [220, 221, 235],
        ...,
        [231, 237, 244],
        [232, 238, 245],
        [233, 239, 246]